In [1]:
#@title Mount google drive
from google.colab import drive
drive.mount('/content/drive')
from sys import version_info
python_version = f"{version_info.major}.{version_info.minor}"

Mounted at /content/drive


In [7]:
#@title Input protein sequence, then hit `Runtime` -> `Run all`

input_dir = '/content/drive/MyDrive/alphafold2/intersection.fasta' #@param {type:"string"}
result_dir = '/content/drive/MyDrive/alphafold2/result' #@param {type:"string"}

# number of models to use
#@markdown ---
#@markdown ### Advanced settings
msa_mode = "MMseqs2 (UniRef+Environmental)" #@param ["MMseqs2 (UniRef+Environmental)", "MMseqs2 (UniRef only)","single_sequence","custom"]
num_models = 1 #@param [1,2,3,4,5] {type:"raw"}
num_recycles = 3 #@param [1,3,6,12,24,48] {type:"raw"}
stop_at_score = 100 #@param {type:"string"}
#@markdown - early stop computing models once score > threshold (avg. plddt for "structures" and ptmscore for "complexes")
use_custom_msa = False
num_relax = 0 #@param [0, 1, 5] {type:"raw"}
use_amber = num_relax > 0
relax_max_iterations = 200 #@param [0,200,2000] {type:"raw"}
use_templates = False #@param {type:"boolean"}
do_not_overwrite_results = True #@param {type:"boolean"}
zip_results = False #@param {type:"boolean"}

In [8]:
#@title Install dependencies
%%bash -s $use_amber $use_templates $python_version

set -e

USE_AMBER=$1
USE_TEMPLATES=$2
PYTHON_VERSION=$3

if [ ! -f COLABFOLD_READY ]; then
  # install dependencies
  # We have to use "--no-warn-conflicts" because colab already has a lot preinstalled with requirements different to ours
  pip install -q --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold"
  if [ -n "${TPU_NAME}" ]; then
    pip install -q --no-warn-conflicts -U dm-haiku==0.0.10 jax==0.3.25
  fi
  ln -s /usr/local/lib/python3.*/dist-packages/colabfold colabfold
  ln -s /usr/local/lib/python3.*/dist-packages/alphafold alphafold
  touch COLABFOLD_READY
fi

# Download params (~1min)
python -m colabfold.download

# setup conda
if [ ${USE_AMBER} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f CONDA_READY ]; then
    wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Miniconda3-latest-Linux-x86_64.sh
    conda config --set auto_update_conda false
    touch CONDA_READY
  fi
fi
# setup template search
if [ ${USE_TEMPLATES} == "True" ] && [ ! -f HH_READY ]; then
  conda install -y -q -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 python="${PYTHON_VERSION}" 2>&1 1>/dev/null
  touch HH_READY
fi
# setup openmm for amber refinement
if [ ${USE_AMBER} == "True" ] && [ ! -f AMBER_READY ]; then
  conda install -y -q -c conda-forge openmm=8.2.0 python="${PYTHON_VERSION}" pdbfixer 2>&1 1>/dev/null
  touch AMBER_READY
fi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.9/373.9 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 116.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 8.2 MB/s eta 0:00:00


In [9]:
#@title Run Prediction

import sys

from colabfold.batch import get_queries, run
from colabfold.download import default_data_dir
from colabfold.utils import setup_logging
from pathlib import Path

# For some reason we need that to get pdbfixer to import
if use_amber and f"/usr/local/lib/python{python_version}/site-packages/" not in sys.path:
    sys.path.insert(0, f"/usr/local/lib/python{python_version}/site-packages/")

setup_logging(Path(result_dir).joinpath("log.txt"))

queries, is_complex = get_queries(input_dir)
run(
    queries=queries,
    result_dir=result_dir,
    use_templates=use_templates,
    num_relax=num_relax,
    relax_max_iterations=relax_max_iterations,
    msa_mode=msa_mode,
    model_type="auto",
    num_models=num_models,
    num_recycles=num_recycles,
    model_order=[1, 2, 3, 4, 5],
    is_complex=is_complex,
    data_dir=default_data_dir,
    keep_existing_results=do_not_overwrite_results,
    rank_by="auto",
    pair_mode="unpaired+paired",
    stop_at_score=stop_at_score,
    zip_results=zip_results,
    user_agent="colabfold/google-colab-batch",
)

2025-05-12 21:18:14,634 Running on GPU
2025-05-12 21:18:14,970 Found 5 citations for tools or databases
2025-05-12 21:18:14,972 Query 1/13: intersection_protein_seq_1 (length 53)


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:01 remaining: 00:00]


2025-05-12 21:18:29,296 Padding length to 63
2025-05-12 21:18:56,245 alphafold2_ptm_model_1_seed_000 recycle=0 pLDDT=88.1 pTM=0.62
2025-05-12 21:19:01,087 alphafold2_ptm_model_1_seed_000 recycle=1 pLDDT=87 pTM=0.61 tol=0.392
2025-05-12 21:19:05,948 alphafold2_ptm_model_1_seed_000 recycle=2 pLDDT=86.4 pTM=0.604 tol=0.354
2025-05-12 21:19:10,813 alphafold2_ptm_model_1_seed_000 recycle=3 pLDDT=85.2 pTM=0.592 tol=0.417
2025-05-12 21:19:10,814 alphafold2_ptm_model_1_seed_000 took 41.5s (3 recycles)
2025-05-12 21:19:10,833 reranking models by 'plddt' metric
2025-05-12 21:19:10,833 rank_001_alphafold2_ptm_model_1_seed_000 pLDDT=85.2 pTM=0.592
2025-05-12 21:19:11,190 Query 2/13: intersection_protein_seq_8 (length 65)


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:00 remaining: 00:00]


2025-05-12 21:19:13,334 Padding length to 75
2025-05-12 21:19:41,199 alphafold2_ptm_model_1_seed_000 recycle=0 pLDDT=54.6 pTM=0.311
2025-05-12 21:19:46,630 alphafold2_ptm_model_1_seed_000 recycle=1 pLDDT=52.6 pTM=0.317 tol=3.29
2025-05-12 21:19:52,080 alphafold2_ptm_model_1_seed_000 recycle=2 pLDDT=54.1 pTM=0.322 tol=3.08
2025-05-12 21:19:57,554 alphafold2_ptm_model_1_seed_000 recycle=3 pLDDT=52.9 pTM=0.324 tol=4.62
2025-05-12 21:19:57,562 alphafold2_ptm_model_1_seed_000 took 44.2s (3 recycles)
2025-05-12 21:19:57,585 reranking models by 'plddt' metric
2025-05-12 21:19:57,586 rank_001_alphafold2_ptm_model_1_seed_000 pLDDT=52.9 pTM=0.324
2025-05-12 21:19:58,111 Query 3/13: intersection_protein_seq_13 (length 82)


...

COMPLETE: 100%|██████████| 150/150 [elapsed: 00:01 remaining: 00:00]


2025-05-12 21:29:26,022 Padding length to 278
2025-05-12 21:30:19,037 alphafold2_ptm_model_1_seed_000 recycle=0 pLDDT=79.3 pTM=0.532
2025-05-12 21:30:43,014 alphafold2_ptm_model_1_seed_000 recycle=1 pLDDT=79.5 pTM=0.54 tol=6.68
2025-05-12 21:31:06,633 alphafold2_ptm_model_1_seed_000 recycle=2 pLDDT=79.1 pTM=0.547 tol=5.98
2025-05-12 21:31:30,353 alphafold2_ptm_model_1_seed_000 recycle=3 pLDDT=79.1 pTM=0.555 tol=1.99
2025-05-12 21:31:30,354 alphafold2_ptm_model_1_seed_000 took 124.3s (3 recycles)
2025-05-12 21:31:30,483 reranking models by 'plddt' metric
2025-05-12 21:31:30,484 rank_001_alphafold2_ptm_model_1_seed_000 pLDDT=79.1 pTM=0.555
2025-05-12 21:31:31,270 Query 13/13: intersection_protein_seq_9 (length 367)


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:01 remaining: 00:00]


2025-05-12 21:33:06,249 alphafold2_ptm_model_1_seed_000 recycle=0 pLDDT=85.6 pTM=0.863
2025-05-12 21:33:42,651 alphafold2_ptm_model_1_seed_000 recycle=1 pLDDT=86 pTM=0.867 tol=0.959
2025-05-12 21:34:18,998 alphafold2_ptm_model_1_seed_000 recycle=2 pLDDT=85.8 pTM=0.861 tol=0.492
2025-05-12 21:34:55,416 alphafold2_ptm_model_1_seed_000 recycle=3 pLDDT=85.8 pTM=0.862 tol=0.375
2025-05-12 21:34:55,417 alphafold2_ptm_model_1_seed_000 took 183.6s (3 recycles)
2025-05-12 21:34:55,555 reranking models by 'plddt' metric
2025-05-12 21:34:55,555 rank_001_alphafold2_ptm_model_1_seed_000 pLDDT=85.8 pTM=0.862
2025-05-12 21:34:56,081 Done


{'rank': [['rank_001_alphafold2_ptm_model_1_seed_000'],
  ['rank_001_alphafold2_ptm_model_1_seed_000'],
  ['rank_001_alphafold2_ptm_model_1_seed_000'],
  ['rank_001_alphafold2_ptm_model_1_seed_000'],
  ['rank_001_alphafold2_ptm_model_1_seed_000'],
  ['rank_001_alphafold2_ptm_model_1_seed_000'],
  ['rank_001_alphafold2_ptm_model_1_seed_000'],
  ['rank_001_alphafold2_ptm_model_1_seed_000'],
  ['rank_001_alphafold2_ptm_model_1_seed_000'],
  ['rank_001_alphafold2_ptm_model_1_seed_000'],
  ['rank_001_alphafold2_ptm_model_1_seed_000'],
  ['rank_001_alphafold2_ptm_model_1_seed_000'],
  ['rank_001_alphafold2_ptm_model_1_seed_000']],
 'metric': [[{'mean_plddt': 85.25,
    'ptm': 0.591796875,
    'print_line': ' pLDDT=85.2 pTM=0.592'}],
  [{'mean_plddt': 52.90625,
    'ptm': 0.32373046875,
    'print_line': ' pLDDT=52.9 pTM=0.324'}],
  [{'mean_plddt': 90.5,
    'ptm': 0.53662109375,
    'print_line': ' pLDDT=90.5 pTM=0.537'}],
  [{'mean_plddt': 88.3125,
    'ptm': 0.75244140625,
    'print_line'